# Question 2

### Implementation details:
1. The program first reads the input using pandas' read_csv() function and the dataset is divided into training data(80%) and test data(20%).
2. We then assign X and Y as attibutes and label respectively for the decision tree.
3. We will be needing the mean and standard deviation for each attribute for the gaussian probability density calculations so we precompute the means and standard deviations.
4. We then compute the likelihood of each label by using the naive bayes assumption i.e. we multiply likelihood for each attribute.
5. We will get two likelihoods, one for label label yes and one for label no.
6. The one with greater likelihood is our prediction.
7. The last step is to calculate the performance of the model for validation data. We report accuracy, precision, recall and F1 score.

In [3]:
import pandas as pd
import numpy as np
import sys

csv_path = 'LoanDataset/data.csv'#raw_input("Enter path to input CSV file: ")
dataset = pd.read_csv(csv_path, header=None)

dataset = dataset.iloc[1:]
dataset.drop(dataset.columns[[0]], axis=1, inplace=True)

#split data into train data and validation data
splitted = np.split(dataset, [int(.8 * len(dataset.index))])
train_data = splitted[0]
validation_data = splitted[1]

In [4]:
X = dataset.keys()[[0,1,2,3,4,5,6,7,9,10,11,12]]
Y = dataset.keys()[8]
print X, Y

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13], dtype='int64') 9


In [5]:
def is_numeric(att):
    if att in dataset.keys()[[0,1,2,4,5,7]]:
        return True
    else:
        return False

mean = {}
sd = {}

for x in X:
#     if is_numeric(x):
    accept = train_data[train_data[Y] == 1][x]
    reject = train_data[train_data[Y] == 0][x]

    mean[x] = [np.mean(reject), np.mean(accept)]
    sd[x] = [np.std(reject), np.std(accept)]

    print x, mean[x], sd[x]

1 [45.233590138674884, 45.387005649717516] [11.430164976692916, 11.417844620576561]
2 [19.99075500770416, 20.15536723163842] [11.453311437371603, 11.326990764558571]
3 [66.88289676425269, 143.80790960451978] [40.886326317539826, 31.916776394308027]
4 [93118.3657935285, 93175.37005649718] [2295.8496284439493, 1769.6717581008547]
5 [2.372265023112481, 2.6073446327683616] [1.1495700076506965, 1.125489101407227]
6 [1.7339137134052387, 3.9105084745762717] [1.571073582652252, 2.0925608810471044]
7 [1.8391371340523883, 2.2401129943502824] [0.8382294025332762, 0.760222974056378]
8 [52.10508474576271, 103.590395480226] [92.84300111209942, 161.95465299048163]
10 [0.10138674884437596, 0.12429378531073447] [0.30184014975338086, 0.3299164140261341]
11 [0.03605546995377504, 0.3050847457627119] [0.18642819808223077, 0.46044331210868217]
12 [0.6049306625577812, 0.6271186440677966] [0.48886558076344977, 0.48357093619278324]
13 [0.2942989214175655, 0.2966101694915254] [0.45572696460712425, 0.45676315180

In [6]:
# import scipy.stats
import math

y1_filtered = train_data[train_data[Y] == 1]
y0_filtered = train_data[train_data[Y] == 0]
p_y1 = float(len(y1_filtered)) / len(train_data)
p_y0 = 1 - p_y1
    
def predict(row):
    
    p_accept = 1
    p_reject = 1
    
    for x in X:
        
        row_val = row[x].values[0]
        
#         filtered = train_data[train_data[x] == row_val]
#         p_x = float(len(filtered)) / len(train_data)
    
#         p_XbyY = scipy.stats.norm(mean[x][1], sd[x][1]).pdf(0)
#         p_accept *= p_XbyY * p_y1 / p_x

#         p_XbyY = scipy.stats.norm(mean[x][0], sd[x][0]).pdf(0)
#         p_reject *= p_XbyY * p_y0 / p_x

#         if is_numeric(x):
        exponent = math.exp(-(math.pow(row_val-mean[x][0],2)/(2*math.pow(sd[x][0],2))))
        p_reject *= ( 1 / (math.sqrt(2*math.pi)*sd[x][0]) ) * exponent

        exponent = math.exp(-(math.pow(row_val-mean[x][1],2)/(2*math.pow(sd[x][1],2))))
        p_accept *= ( 1 / (math.sqrt(2*math.pi)*sd[x][1]) ) * exponent
            
#         else:

#             filtered = y1_filtered[y1_filtered[x] == row_val]
#             required = len(filtered)
            
#             p_accept *= float(required) / len(y1_filtered)

#             filtered = y0_filtered[y0_filtered[x] == row_val]
#             required = len(filtered)
            
#             p_reject *= float(required) / len(y0_filtered)
    
#     p_accept *= p_y1
#     p_reject *= p_y0
    
#     print p_accept, p_reject
    
    if p_accept > p_reject:
        return 1
    else:
        return 0

In [8]:
def calculate_performance(validation_data):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for i in range(len(validation_data)):
        row = validation_data.iloc[[i]][Y]
        row = row.tolist()
        row = row[0]
        if predict(validation_data.iloc[[i]]) == 1:
            if row == 1:
                TP += 1
            else:
                FP += 1
        else:
            if row == 0:
                TN += 1
            else:
                FN += 1

    accuracy = float(TP + TN) / (TP + TN + FP + FN)
    
    if TP + FP == 0:
        precision = 0
    else:
        precision = float(TP) / (TP + FP)
        
    if TP + FN == 0:
        recall = 0
    else:
        recall = float(TP) / (TP + FN)
    
    if precision == 0 or recall == 0:
        F1measure = 0
    else:
        F1measure = 2.0 / ( (1/recall) + (1/precision) )
    
#     print TP, TN, FP, FN
    print ("Validation Results:\n")
    print ("accuracy = " + str(accuracy))
    print ("precision = " + str(precision))
    print ("recall = " + str(recall))
    print ("F1 measure = " + str(F1measure))


calculate_performance(validation_data)

Validation Results:

accuracy = 0.877777777778
precision = 0.401315789474
recall = 0.7625
F1 measure = 0.525862068966


#### Observations: 
We have got good accuracy but precision is low